In [1]:
import cdflib
import spacepy.pycdf as pycdf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# File paths (update these to your actual file paths)
files = {
    'L1_AUX': 'cdf_data/20250621/AL1_ASW91_L1_AUX_20250621_UNP_9999_999999_V01.cdf',
    'L1_TH1': 'cdf_data/20250621/AL1_ASW91_L1_TH1_20250621_UNP_9999_999999_V01.cdf',
    'L1_TH2': 'cdf_data/20250621/AL1_ASW91_L1_TH2_20250621_UNP_9999_999999_V01.cdf',
    'L2_BLK': 'cdf_data/20250621/AL1_ASW91_L2_BLK_20250621_UNP_9999_999999_V02.cdf',
    'L2_TH1': 'cdf_data/20250621/AL1_ASW91_L2_TH1_20250621_UNP_9999_999999_V02.cdf',
    'L2_TH2': 'cdf_data/20250621/AL1_ASW91_L2_TH2_20250621_UNP_9999_999999_V02.cdf'
}

In [2]:
!pip install cupy-cuda12x
!pip install cudf-cu12 --extra-index-url=https://pypi.nvidia.com



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached fastrlock-0.8.3-cp311-cp311-win_amd64.whl.metadata (7.9 kB)
   ---------------------------------------- 0.0/82.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/82.2 MB 682.7 kB/s eta 0:02:01
   ---------------------------------------- 0.0/82.2 MB 330.3 kB/s eta 0:04:09
   ---------------------------------------- 0.1/82.2 MB 550.5 kB/s eta 0:02:30
   ---------------------------------------- 0.1/82.2 MB 731.4 kB/s eta 0:01:53
   ---------------------------------------- 0.1/82.2 MB 774.0 kB/s eta 0:01:46
   ---------------------------------------- 0.2/82.2 MB 1.1 MB/s eta 0:01:12
   ---------------------------------------- 0.3/82.2 MB 1.1 MB/s eta 0:01:15
   ---------------------------------------- 0.4/82.2 MB 1.2 MB/s eta 0:01:08
   ---------------------------------------- 0.5/82.2 MB 1.3 MB/s eta 0:01:01
   ---------------------------------------- 0.6/82.2 MB 1.5 MB/s eta 0:00:56
   ---------------------------------------- 0.6/82.2 MB 1.5 MB/s eta 0:00:5

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [68 lines of output]
      INFO:nvidia-stub:Testing wheel cudf_cu12-24.10.1-cp310-cp310-manylinux_2_24_aarch64.manylinux_2_28_aarch64.whl against tag cp310-cp310-manylinux_2_28_aarch64
      INFO:nvidia-stub:Testing wheel cudf_cu12-24.10.1-cp310-cp310-manylinux_2_24_aarch64.manylinux_2_28_aarch64.whl against tag cp310-cp310-manylinux_2_24_aarch64
      INFO:nvidia-stub:Testing wheel cudf_cu12-24.10.1-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl against tag cp310-cp310-manylinux_2_24_x86_64
      INFO:nvidia-stub:Testing wheel cudf_cu12-24.10.1-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl against tag cp310-cp310-manylinux_2_28_x86_64
      INFO:nvidia-stub:Testing wheel cudf_cu12-24.10.1-cp311-cp311-manylinux_2_24_aarch64.manylinux_2_28_aarch64.whl against tag cp311-cp311-manylinux_2_28_aarch64
      INFO:nvidia-stub:Testing whee

In [1]:
!pip install cdflib cupy-cuda12x cudf-cu12 cuml-cu12 --extra-index-url=https://pypi.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Using cached cudf_cu12-24.10.1.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [68 lines of output]
      INFO:nvidia-stub:Testing wheel cudf_cu12-24.10.1-cp310-cp310-manylinux_2_24_aarch64.manylinux_2_28_aarch64.whl against tag cp310-cp310-manylinux_2_24_aarch64
      INFO:nvidia-stub:Testing wheel cudf_cu12-24.10.1-cp310-cp310-manylinux_2_24_aarch64.manylinux_2_28_aarch64.whl against tag cp310-cp310-manylinux_2_28_aarch64
      INFO:nvidia-stub:Testing wheel cudf_cu12-24.10.1-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl against tag cp310-cp310-manylinux_2_28_x86_64
      INFO:nvidia-stub:Testing wheel cudf_cu12-24.10.1-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl against tag cp310-cp310-manylinux_2_24_x86_64
      INFO:nvidia-stub:Testing wheel cudf_cu12-24.10.1-cp311-cp311-manylinux_2_24_aarch64.manylinux_2_28_aarch64.whl against tag cp311-cp311-manylinux_2_24_aarch64
      INFO:nvidia-stub:Testing whee

In [2]:
import cdflib
import numpy as np
import cupy as cp
import cudf
from cuml.impute import SimpleImputer
from datetime import datetime, timedelta

# ---- Step 1: Load CDF File on CPU ----
def load_cdf_gpu(file_path, max_length=10_000_000):
    try:
        cdf = cdflib.CDF(file_path)
        info = cdf.cdf_info()
        variables = info.rVariables + info.zVariables

        data = {}
        for var in variables:
            try:
                values = cdf.varget(var)
            except Exception as e:
                print(f"⚠️ Skipping {var}: {e}")
                continue

            if hasattr(values, '__len__') and len(values) > max_length:
                print(f"⚠️ Skipping {var}: too large ({len(values)} elements)")
                continue

            fillval = cdf.varattsget(var).get('FILLVAL', -1.0e31)

            if np.isscalar(values):
                continue

            values = np.array(values, dtype=np.float32)
            values[values == fillval] = np.nan

            if values.ndim > 1 and var not in ['angle_tha1', 'angle_tha2']:
                values = np.nanmean(values, axis=tuple(range(1, values.ndim)))

            data[var] = cp.asarray(values)  # Push to GPU

        return data

    except Exception as e:
        print(f"❌ Error loading {file_path}: {e}")
        return None

# ---- Step 2: Convert to cuDF DataFrame (GPU) ----
def convert_to_cudf(data_dict):
    df = cudf.DataFrame()
    for key, arr in data_dict.items():
        try:
            df[key] = cudf.Series(arr)
        except Exception as e:
            print(f"⚠️ Skipping {key}: {e}")
    return df

# ---- Step 3: Impute Missing Values using cuML ----
def impute_gpu(df):
    imputer = SimpleImputer(strategy="mean")
    try:
        return imputer.fit_transform(df)
    except Exception as e:
        print(f"⚠️ Imputation failed: {e}")
        return df

# ---- Step 4: Epoch Conversion (on CPU) ----
def epoch_to_datetime(epoch):
    j2000 = datetime(2000, 1, 1, 11, 58, 55, 816000)
    return [j2000 + timedelta(seconds=t / 1e9) for t in epoch]

# ---- Step 5: Full Pipeline ----
def process_cdf_file(file_path):
    print(f"📂 Processing: {file_path}")
    data_dict = load_cdf_gpu(file_path)
    if data_dict is None:
        return None

    # Optional: convert epoch if exists
    for k in ['epoch_for_cdf', 'epoch_for_cdf_mod']:
        if k in data_dict:
            epoch = cp.asnumpy(data_dict[k])
            time_col = epoch_to_datetime(epoch)
            del data_dict[k]
            data_dict['time'] = cp.asarray(time_col)

    df_gpu = convert_to_cudf(data_dict)
    df_imputed = impute_gpu(df_gpu)
    return df_imputed


c:\Users\harsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


ModuleNotFoundError: No module named 'cudf'

In [ ]:
# Replace with your actual file path
file_path = "cdf_data/20250621/AL1_ASW91_L1_AUX_20250621_UNP_9999_999999_V01.cdf"

# Run the pipeline
gpu_df = process_cdf_file(file_path)

# Show the first few rows (on GPU)
print(gpu_df.head())


In [1]:
import cdflib
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# --- Function to load CDF data ---
from sklearn.impute import SimpleImputer

def load_cdf(file_path):
    try:
        cdf = cdflib.CDF(file_path)
        info = cdf.cdf_info()
        variables = info.rVariables + info.zVariables

        data = {}
        for var in variables:
            values = cdf.varget(var)
            fillval = cdf.varattsget(var).get('FILLVAL', -1.0e31)

            if np.isscalar(values):
                continue

            values = np.array(values)

            # Convert to float to support np.nan
            if np.issubdtype(values.dtype, np.integer):
                values = values.astype(np.float64)

            values[values == fillval] = np.nan

            # Average multi-dimensional data
            if values.ndim > 1 and var not in ['sun_angle_tha1', 'sun_angle_tha2', 'angle_tha1', 'angle_tha2']:
                values = np.nanmean(values, axis=tuple(range(1, values.ndim)))

            # Replace NaNs using sklearn imputer
            if np.any(np.isnan(values)):
                values = np.array(values).reshape(-1, 1)
                imputer = SimpleImputer(strategy='mean')  # You can use 'median', 'most_frequent', etc.
                values = imputer.fit_transform(values).ravel()

            data[var] = values

        return data

    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None



# --- Function to convert epoch to datetime ---
def epoch_to_datetime(epoch):
    j2000 = datetime(2000, 1, 1, 11, 58, 55, 816000)
    return [j2000 + timedelta(seconds=t / 1e9) for t in epoch]

# --- Placeholder file paths ---
files = {
    "L1_AUX": "cdf_data/20250621/AL1_ASW91_L1_AUX_20250621_UNP_9999_999999_V01.cdf",
    "L1_TH1": "cdf_data/20250621/AL1_ASW91_L1_TH1_20250621_UNP_9999_999999_V01.cdf",
    "L1_TH2": "cdf_data/20250621/AL1_ASW91_L1_TH2_20250621_UNP_9999_999999_V01.cdf",
    "L2_BLK": "cdf_data/20250621/AL1_ASW91_L2_BLK_20250621_UNP_9999_999999_V02.cdf",
    "L2_TH1": "cdf_data/20250621/AL1_ASW91_L2_TH1_20250621_UNP_9999_999999_V02.cdf",
    "L2_TH2": "cdf_data/20250621/AL1_ASW91_L2_TH2_20250621_UNP_9999_999999_V02.cdf"
}

# --- Load all datasets ---
datasets = {key: load_cdf(path) for key, path in files.items()}

# --- Preprocess data ---
data_frames = {}
for key, data in datasets.items():
    if data is None:
        continue

    epoch_var = 'epoch_for_cdf_mod' if key in ['L1_AUX', 'L1_TH1', 'L1_TH2'] else 'epoch_for_cdf'
    if epoch_var in data:
        times = epoch_to_datetime(data[epoch_var])
        df = pd.DataFrame({'time': times})
        for var in data:
            if var != epoch_var and not np.isscalar(data[var]):
                df[var] = data[var]
        df = df.interpolate(method='time').fillna(method='bfill')
        data_frames[key] = df

# --- Compute AHe ratio if applicable ---
for df in data_frames.values():
    if 'proton_density' in df.columns and 'alpha_density' in df.columns:
        df['ahe'] = df['alpha_density'] / (df['proton_density'] + 1e-10)

# --- Threshold calculation ---
def calculate_threshold(flux, flux_uncer, window_size=100, k=2, q=99, t_min=1e-2):
    epsilon = 1e-10
    flux_log = np.log10(flux + epsilon)
    flux_uncer_log = flux_uncer / (flux + epsilon) / np.log(10)

    p_q = np.percentile(flux_log[-window_size:], q)
    p_25 = np.percentile(flux_log[-window_size:], 25)
    p_75 = np.percentile(flux_log[-window_size:], 75)
    iqr = p_75 - p_25

    threshold_log = p_q + k * (iqr + flux_uncer_log[-1])
    return max(10 ** threshold_log, t_min)


Error loading cdf_data/20250621/AL1_ASW91_L1_TH1_20250621_UNP_9999_999999_V01.cdf: Unable to allocate output buffer.


c:\Users\harsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:558: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
c:\Users\harsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:558: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
c:\Users\harsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:558: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
c:\Users\harsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:558: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for im

In [ ]:
# Apply threshold to L1_TH1 and L1_TH2
for key in ['L1_TH1', 'L1_TH2']:
    if key in data_frames and 'integrated_flux_mod' in data_frames[key]:
        flux = data_frames[key]['integrated_flux_mod']
        flux_uncer = 0.1 * flux  # Proxy for uncertainty
        thresholds = [calculate_threshold(flux[:i+1], flux_uncer[:i+1]) for i in range(len(flux))]
        data_frames[key]['threshold'] = thresholds
        data_frames[key]['is_cme'] = (flux > data_frames[key]['threshold']) & (data_frames[key]['ahe'] > 0.1)

# Visualization - 6 plots
plt.figure(figsize=(15, 20))

# 1. L1_AUX: Proton Density vs. Time
plt.subplot(3, 2, 1)
if 'L1_AUX' in data_frames and 'proton_density' in data_frames['L1_AUX']:
    plt.plot(data_frames['L1_AUX']['time'], data_frames['L1_AUX']['proton_density'], label='Proton Density')
    plt.xlabel('Time')
    plt.ylabel('Proton Density')
    plt.title('L1_AUX: Proton Density')
    plt.legend()
else:
    plt.text(0.5, 0.5, 'L1_AUX Data Unavailable', ha='center', va='center')
    plt.title('L1_AUX: Proton Density')

# 2. L1_TH1: Integrated Flux vs. Time
plt.anaghplt.subplot(3, 2, 2)
if 'L1_TH1' in data_frames and 'integrated_flux_mod' in data_frames['L1_TH1']:
    plt.plot(data_frames['L1_TH1']['time'], data_frames['L1_TH1']['integrated_flux_mod'], label='Integrated Flux')
    plt.plot(data_frames['L1_TH1']['time'], data_frames['L1_TH1']['threshold'], label='Threshold', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Flux')
    plt.title('L1_TH1: Integrated Flux')
    plt.legend()
else:
    plt.text(0.5, 0.5, 'L1_TH1 Data Unavailable', ha='center', va='center')
    plt.title('L1_TH1: Integrated Flux')

# 3. L1_TH2: Integrated Flux vs. Time
plt.subplot(3, 2, 3)
if 'L1_TH2' in data_frames and 'integrated_flux_mod' in data_frames['L1_TH2']:
    plt.plot(data_frames['L1_TH2']['time'], data_frames['L1_TH2']['integrated_flux_mod'], label='Integrated Flux')
    plt.plot(data_frames['L1_TH2']['time'], data_frames['L1_TH2']['threshold'], label='Threshold', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Flux')
    plt.title('L1_TH2: Integrated Flux')
    plt.legend()
else:
    plt.text(0.5, 0.5, 'L1_TH2 Data Unavailable', ha='center', va='center')
    plt.title('L1_TH2: Integrated Flux')

# 4. L2_BLK: Angle XGSE vs. Time
plt.subplot(3, 2, 4)
if 'L2_BLK' in data_frames and 'angle_xgse' in data_frames['L2_BLK']:
    plt.plot(data_frames['L2_BLK']['time'], data_frames['L2_BLK']['angle_xgse'], label='Angle XGSE')
    plt.xlabel('Time')
    plt.ylabel('Angle XGSE (°)')
    plt.title('L2_BLK: Angle XGSE')
    plt.legend()
else:
    plt.text(0.5, 0.5, 'L2_BLK Data Unavailable', ha='center', va='center')
    plt.title('L2_BLK: Angle XGSE')

# 5. L2_TH1: THA-1 Spectral Counts vs. Time
plt.subplot(3, 2, 5)
if 'L2_TH1' in datasets and 'THA-1_spec' in datasets['L2_TH1']:
    mean_spec = np.nanmean(datasets['L2_TH1']['THA-1_spec'], axis=(1, 2, 3))
    plt.plot(data_frames['L2_TH1']['time'][:len(mean_spec)], mean_spec, label='Mean Spectral Counts')
    plt.xlabel('Time')
    plt.ylabel('Mean Counts')
    plt.title('L2_TH1: THA-1 Spectral Counts')
    plt.legend()
else:
    plt.text(0.5, 0.5, 'L2_TH1 Data Unavailable', ha='center', va='center')
    plt.title('L2_TH1: THA-1 Spectral Counts')

# 6. L2_TH2: THA-2 Spectral Counts vs. Time
plt.subplot(3, 2, 6)
if 'L2_TH2' in datasets and 'THA-2_spec' in datasets['L2_TH2']:
    mean_spec = np.nanmean(datasets['L2_TH2']['THA-2_spec'], axis=(1, 2))
    plt.plot(data_frames['L2_TH2']['time'][:len(mean_spec)], mean_spec, label='Mean Spectral Counts')
    plt.xlabel('Time')
    plt.ylabel('Mean Counts')
    plt.title('L2_TH2: THA-2 Spectral Counts')
    plt.legend()
else:
    plt.text(0.5, 0.5, 'L2_TH2 Data Unavailable', ha='center', va='center')
    plt.title('L2_TH2: THA-2 Spectral Counts')

plt.tight_layout()
plt.show()

# Print CME detection summary
for key in ['L1_TH1', 'L1_TH2']:
    if key in data_frames and 'is_cme' in data_frames[key]:
        cme_count = data_frames[key]['is_cme'].sum()
        print(f"{key} CME Detections: {cme_count} events on 2025-06-21")
    else:
        print(f"{key} CME Detections: No data available")